# Boardroom Network Connectivity

In [1]:
import pandas as pd
import numpy as np

In [2]:
!nvidia-smi

Mon Feb 24 18:58:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
boardex = pd.read_csv('boardex.csv')
#crsp = pd.read_csv('crsp.csv')
#linking_table = pd.read_csv('boardex_crsp_comp_link.csv')

In [5]:
boardex

,associationtype,companyid,overlapyearstart_int,boardname,companyname,directorname,role,associatedrole,conncompanyorgtype,boardid,directorid,roletitle,roleboardposition,roleedflag,startcompanydatestartrole,overlapyearend_int,startcompanydateendrole,conncompanydatestartrole,conncompanydateendrole,orgtype
0,Listed Org,1094770,2004,1 800 CONTACTS INC (De-listed 09/2007),IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...,Steve Newman,Executive Officer (Non-Brd),Independent NED (Brd) (SD),Quoted,3,341621,Executive Officer,Non-Brd,NaN,2002-01-02,2007.0,2007-09-07,2004-04-05,2009-07-28,Quoted
1,Listed Org,13761,1998,1 800 CONTACTS INC (De-listed 09/2007),GRANDVISION (De-listed 03/2004),Edward Butler,Independent Director (Brd) (SD),Chairman (Brd) (SD),Quoted,3,340871,Independent Director,Brd,SD,1998-01-01,1998.0,2007-09-07,1997-01-02,1998-12-31,Quoted
2,Listed Org,28148,2006,1 800 CONTACTS INC (De-listed 09/2007),SITEL CORP (De-listed 01/2007),Steve Key,Independent Director (Brd) (SD),Independent Director (Brd) (SD),Quoted,3,59796,Independent Director,Brd,SD,2005-07-28,2007.0,2007-09-07,2006-08-04,2007-01-31,Quoted
3,Listed Org,40757,2005,1 800 CONTACTS INC (De-listed 09/2007),GREENHILL & CO INC (De-listed 12/2023),Steve Key,Independent Director (Brd) (SD),Independent Director (Brd) (SD),Quoted,3,59796,Independent Director,Brd,SD,2005-07-28,2007.0,2007-09-07,2004-05-01,2020-04-23,Quoted
4,Listed Org,16506,2023,1-800-FLOWERS.COM INC,INTERNATIONAL BUSINESS MACHINES (IBM) CORP,Christina Shim,Director - SD (Brd) (SD),VP/Head of Division (Non-Brd),Quoted,6,2779779,Director - SD,Brd,SD,2023-12-14,2024.0,9000-01-01,2021-11-01,2024-05-02,Quoted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008301,Listed Org,2006211,2023,City Therapeutics Inc,KARYOPHARM THERAPEUTICS INC,Professor Barry Greene,Director - SD (Brd) (SD),Lead Independent Director (Brd) (SD),Quoted,3907518,345845,Director - SD,Brd,SD,1900-01-01,NaN,9000-01-01,2015-01-01,9000-01-01,Private
1008302,Listed Org,2360912,2024,City Therapeutics Inc,VOYAGER THERAPEUTICS INC,Doctor Phil Zamore,Scientific Advisor (Non-Brd),Scientific Advisor (Non-Brd),Quoted,3907518,1273575,Scientific Advisor,Non-Brd,NaN,1900-01-01,NaN,9000-01-01,2020-05-01,9000-01-01,Private
1008303,Listed Org,2147864,2024,City Therapeutics Inc,PROQR THERAPEUTICS NV,Doctor Phil Zamore,Scientific Advisor (Non-Brd),Scientific Advisor (Non-Brd),Quoted,3907518,1273575,Scientific Advisor,Non-Brd,NaN,1900-01-01,NaN,9000-01-01,2017-08-28,9000-01-01,Private
1008304,Listed Org,1986196,2024,RuggedEdge Inc,MCLOUD TECHNOLOGIES CORP (Universal mCloud Cor...,Vincent Higgins,CEO (Brd) (ED),Division President (Non-Brd),Quoted,3915440,881021,CEO,Brd,ED,2024-04-01,2024.0,9000-01-01,2022-01-26,2024-05-22,Private


In [6]:
# Rename columns
boardex = boardex.rename(columns={
    'overlapyearstart_int': 'start_year',
    'overlapyearend_int': 'end_year',
    'boardname': 'company1',
    'companyname': 'company2',
    'directorname': 'person',
    'associatedrole': 'company2_role',
    'roleboardposition': 'company1_role',
    'conncompanyorgtype': 'company2_orgtype',
    'orgtype': 'company1_orgtype',
    'boardid': 'company1_id',
    'companyid': 'company2_id'
})

In [7]:
# Only keep if Board or Non-Board
boardex['company2_role'] = boardex['company2_role'].astype(str)

boardex['company2_role'] = np.where(
    boardex['company2_role'].str.contains(r'\(Brd\)', case=False, na=False),
    'Brd',
    'Non-Brd'
)
boardex = boardex[boardex['company1_role'].isin(['Brd', 'Non-Brd'])]

In [8]:
# Drop columns
boardex.drop(columns=['associationtype','roletitle', 'roleedflag', 'role', 'startcompanydatestartrole', 'startcompanydateendrole', 'conncompanydatestartrole', 'conncompanydateendrole'], inplace=True)

In [9]:
new_order = ['start_year', 'end_year', 'company1', 'company2', 'person',
             'company1_role', 'company2_role', 'company1_orgtype', 'company2_orgtype',
             'company1_id', 'company2_id']
boardex = boardex[new_order]

In [10]:
boardex.head(5)

,start_year,end_year,company1,company2,person,company1_role,company2_role,company1_orgtype,company2_orgtype,company1_id,company2_id
0,2004,2007.0,1 800 CONTACTS INC (De-listed 09/2007),IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...,Steve Newman,Non-Brd,Brd,Quoted,Quoted,3,1094770
1,1998,1998.0,1 800 CONTACTS INC (De-listed 09/2007),GRANDVISION (De-listed 03/2004),Edward Butler,Brd,Brd,Quoted,Quoted,3,13761
2,2006,2007.0,1 800 CONTACTS INC (De-listed 09/2007),SITEL CORP (De-listed 01/2007),Steve Key,Brd,Brd,Quoted,Quoted,3,28148
3,2005,2007.0,1 800 CONTACTS INC (De-listed 09/2007),GREENHILL & CO INC (De-listed 12/2023),Steve Key,Brd,Brd,Quoted,Quoted,3,40757
4,2023,2024.0,1-800-FLOWERS.COM INC,INTERNATIONAL BUSINESS MACHINES (IBM) CORP,Christina Shim,Brd,Non-Brd,Quoted,Quoted,6,16506


In [11]:
boardex['company1_role'].value_counts()
boardex['company2_role'].value_counts()

,count
company2_role,
Brd,658309
Non-Brd,335874


In [12]:
# Drop rows where both company1_role and company2_role equal 'Non-brd'
boardex = boardex[~((boardex['company1_role'] == 'Non-brd') & (boardex['company2_role'] == 'Non-brd'))]

In [13]:
# Drop duplicate rows based on company1, company2, and person columns, keeping the first occurrence
boardex = boardex.drop_duplicates(subset=['company1', 'company2', 'person'], keep='first')

In [14]:
boardex['company1_orgtype'].value_counts()

,count
company1_orgtype,
Quoted,542692
Private,81594
Partnership,21603
Government,1725
Charities,430
Clubs,16


In [15]:
# Drop rows where orgtype is 'charity' or 'club'
boardex = boardex[~boardex['company1_orgtype'].isin(['Charities', 'Clubs'])]

In [16]:
boardex['company2_orgtype'].value_counts()

,count
company2_orgtype,
Quoted,647614


In [17]:
boardex_df = boardex

def expand_years(row):
    # If start_year is missing, we cannot expand, so assign None to 'year'
    if pd.isna(row['start_year']):
        new_row = row.copy()
        new_row['year'] = None
        return pd.DataFrame([new_row])

    start_year = int(row['start_year'])

    # If end_year is missing, use 2025; otherwise, convert end_year to int.
    if pd.isna(row['end_year']):
        end_year = 2025
    else:
        end_year = int(row['end_year'])

    # Create a row for each year in the range from start_year to end_year (inclusive)
    new_rows = []
    for year in range(start_year, end_year + 1):
        new_row = row.copy()
        new_row['year'] = year
        new_rows.append(new_row)

    return pd.DataFrame(new_rows)

expanded_df = pd.concat([expand_years(row) for _, row in boardex_df.iterrows()], ignore_index=True)
print(expanded_df.head())


   start_year  end_year                                company1  \
0        2004    2007.0  1 800 CONTACTS INC (De-listed 09/2007)   
1        2004    2007.0  1 800 CONTACTS INC (De-listed 09/2007)   
2        2004    2007.0  1 800 CONTACTS INC (De-listed 09/2007)   
3        2004    2007.0  1 800 CONTACTS INC (De-listed 09/2007)   
4        1998    1998.0  1 800 CONTACTS INC (De-listed 09/2007)   

                                            company2         person  \
0  IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...   Steve Newman   
1  IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...   Steve Newman   
2  IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...   Steve Newman   
3  IFS CONSTRUCTION SERVICES LTD (Advanced Ocular...   Steve Newman   
4                    GRANDVISION (De-listed 03/2004)  Edward Butler   

  company1_role company2_role company1_orgtype company2_orgtype  company1_id  \
0       Non-Brd           Brd           Quoted           Quoted            3   
1       No

In [21]:
expanded_df.to_csv("expanded_df.csv", index=False)

In [18]:
boardex['company1_id'] = boardex['company1_id'].astype(int)

In [19]:
expanded_df['year'] = pd.to_datetime(expanded_df['year'], format='%Y')

In [ ]:
# # USING NETWORK X
# # Make sure to include company id so that can be merged

# import pandas as pd
# import networkx as nx

# # List to hold per-year metrics DataFrames
# results_list = []

# # Get the unique years in sorted order
# years = sorted(expanded_df['year'].unique())

# for yr in years:
#     # Subset the data for the given year
#     df_year = expanded_df[expanded_df['year'] == yr]

#     # Create an undirected graph
#     G = nx.Graph()

#     # Add edges from the data:
#     # Each row represents an edge connecting company1 and company2.
#     # We can also assign a weight to count multiple links.
#     for _, row in df_year.iterrows():
#         comp1 = row['company1']
#         comp2 = row['company2']
#         if G.has_edge(comp1, comp2):
#             G[comp1][comp2]['weight'] += 1
#         else:
#             G.add_edge(comp1, comp2, weight=1)

#     # Calculate network centrality metrics
#     betweenness = nx.betweenness_centrality(G, weight='weight')
#     closeness = nx.closeness_centrality(G)
#     degree = nx.degree_centrality(G)
#     try:
#         eigenvector = nx.eigenvector_centrality(G, weight='weight')
#     except Exception as e:
#         # If eigenvector centrality fails (e.g., due to convergence issues), assign None
#         eigenvector = {node: None for node in G.nodes()}

#     # Create a DataFrame for the metrics in this year
#     df_metrics = pd.DataFrame({
#         'company': list(G.nodes()),
#         'year': yr,
#         'betweenness': [betweenness[node] for node in G.nodes()],
#         'closeness': [closeness[node] for node in G.nodes()],
#         'degree': [degree[node] for node in G.nodes()],
#         'eigenvector': [eigenvector[node] for node in G.nodes()]
#     })

#     results_list.append(df_metrics)

# # Concatenate all yearly DataFrames into one final DataFrame
# metrics_df = pd.concat(results_list, ignore_index=True)

# # Display the resulting metrics DataFrame
# print(metrics_df)

In [20]:
pip install igraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.7 MB/s eta 0:00:00


In [22]:
import pandas as pd
import igraph as ig

# List to hold per-year metrics DataFrames
results_list = []

# Get the unique years in sorted order
years = sorted(expanded_df['year'].unique())

for yr in years:
    # Subset the data for the given year
    df_year = expanded_df[expanded_df['year'] == yr]

    # Get the unique company IDs from both company1_id and company2_id columns
    companies = sorted(set(df_year['company1_id']).union(set(df_year['company2_id'])))

    # Create a mapping from company id to index (for igraph)
    mapping = {company: idx for idx, company in enumerate(companies)}

    # Build an edge dictionary to aggregate weights
    # Using a sorted tuple (company1_id, company2_id) as key to ensure undirected edges
    edge_weights = {}
    for _, row in df_year.iterrows():
        comp1 = row['company1_id']
        comp2 = row['company2_id']
        key = tuple(sorted((comp1, comp2)))
        edge_weights[key] = edge_weights.get(key, 0) + 1

    # Create an igraph Graph and add vertices with the company IDs
    g = ig.Graph()
    g.add_vertices(companies)

    # Prepare edges (as indices) and weights for the graph
    edges = []
    weights = []
    for (u, v), w in edge_weights.items():
        edges.append((mapping[u], mapping[v]))
        weights.append(w)

    # Add edges and set their 'weight' attribute
    g.add_edges(edges)
    g.es['weight'] = weights

    # ---- Calculate network centrality metrics ----
    # Betweenness centrality:
    # In NetworkX the weight is treated as a distance (cost).
    # To mimic that in igraph we use the inverse of the weight (ensuring no weight is zero).
    inv_weights = [1.0/w for w in weights]
    betweenness = g.betweenness(weights=inv_weights)

    # Closeness centrality (unweighted version to match NetworkX default):
    closeness = g.closeness()  # igraph's closeness returns the reciprocal of the sum of distances

    # Degree centrality:
    n = g.vcount()
    degree_counts = g.degree()
    degree = [d/(n-1) if n > 1 else 0 for d in degree_counts]

    # Eigenvector centrality:
    try:
        eigenvector = g.eigenvector_centrality(weights="weight")
    except Exception as e:
        eigenvector = [None] * n  # Assign None if calculation fails

    # Create a DataFrame for the metrics in this year
    df_metrics = pd.DataFrame({
        'company_linking': g.vs['name'],
        'year': yr,
        'betweenness': betweenness,
        'closeness': closeness,
        'degree': degree,
        'eigenvector': eigenvector
    })

    results_list.append(df_metrics)

# Concatenate all yearly DataFrames into one final DataFrame
metrics_df = pd.concat(results_list, ignore_index=True)

# Display the resulting metrics DataFrame
print(metrics_df)


        company_linking       year   betweenness  closeness    degree  \
0                  6930 1960-01-01      0.000000   1.000000  1.000000   
1                 24708 1960-01-01      0.000000   1.000000  1.000000   
2                 80806 1960-01-01      0.000000   1.000000  1.000000   
3                  6930 1961-01-01      0.000000   1.000000  1.000000   
4                 24708 1961-01-01      0.000000   1.000000  1.000000   
...                 ...        ...           ...        ...       ...   
405336          3905369 2025-01-01      0.000000   0.211097  0.000149   
405337          3906385 2025-01-01  38509.770310   0.184001  0.000521   
405338          3906438 2025-01-01  13585.634959   0.174355  0.000223   
405339          3907518 2025-01-01     25.562353   0.202643  0.000819   
405340          3920108 2025-01-01      0.000000   0.153001  0.000074   

         eigenvector  
0       1.000000e+00  
1       1.000000e+00  
2       1.000000e+00  
3       1.000000e+00  
4       

In [23]:
metrics_df.to_csv("metrics_df.csv", index=False)

In [ ]:
metrics_df

In [ ]:
metrics_df[metrics_df['year'] == '2024-01-01 00:00:00']

In [ ]:
pip install igraph

In [24]:
import pandas as pd
import igraph as ig

# Choose the year you want to visualize
year_to_plot = 2018  # change as needed

# Subset the data for the chosen year
df_year = expanded_df[expanded_df['year'] == year_to_plot]

# Get unique companies from both company1_id and company2_id
companies = sorted(set(df_year['company1_id']).union(set(df_year['company2_id'])))
mapping = {company: idx for idx, company in enumerate(companies)}

# Build an edge dictionary to aggregate weights
edge_weights = {}
for _, row in df_year.iterrows():
    comp1 = row['company1_id']
    comp2 = row['company2_id']
    key = tuple(sorted((comp1, comp2)))
    edge_weights[key] = edge_weights.get(key, 0) + 1

# Create an igraph Graph and add vertices (companies)
g = ig.Graph()
g.add_vertices(companies)

# Prepare edges (using indices) and weights
edges = []
weights = []
for (u, v), w in edge_weights.items():
    edges.append((mapping[u], mapping[v]))
    weights.append(w)

g.add_edges(edges)
g.es['weight'] = weights

# Set a layout for visualization (Fruchterman-Reingold layout is often effective)
layout = g.layout("fruchterman_reingold")

# Define visual style parameters
visual_style = {
    "vertex_size": 20,
    "vertex_label": g.vs["name"],
    "edge_width": [max(1, w/2) for w in weights],
    "layout": layout,
    "bbox": (800, 800),
    "margin": 50
}

# Plot the graph
ig.plot(g, **visual_style)


KeyError: 'Attribute does not exist'

### Merge Linking Table

In [ ]:
# # Merge BoardEx with the linking table on 'companyid'
# boardex_linked = boardex.merge(linking_table, on='companyid', how='left')
# boardex_linked.rename(columns={'PERMCO':'permco_company','GVKEY':'gvkey_company'}, inplace=True)

In [ ]:
# # Link the table again for boardID
# boardex_linked2 = boardex_linked.merge(linking_table, left_on='boardid', right_on='companyid', how='left')
# boardex_linked2.rename(columns={'PERMCO':'permco_board','GVKEY':'gvkey_board'}, inplace=True)
# boardex_linked2.drop(columns=['companyid_y'], inplace=True)

In [ ]:
# # Merge CRSP - need to decide if the returns are for boards or companies or both?
# boardex_linked2 = boardex_linked.merge(crsp, left_on='PERMCO_BOARD OR _COMPANY?', right_on='PERMCO', how='left')